<a id='Urbanization_Using_NDBI_top'></a>
# Urbanization Using NDBI
<hr>

## Background
Among the many urbanization indices, the Normalized Difference Built-Up Index (NDBI) is one of the most commonly used. This notebook shows how to use NDBI in the context of the Open Data Cube.

The formula for NDBI for Landsat is as follows:

$$ NDBI =  \frac{(SWIR - NIR)}{(SWIR + NIR)}$$  

Note that for arid environments, the Dry Built-Up Index (DBI) may perform better than NDBI, which struggles with arid environments and some kinds of buildings. DBI requires the TIR band of Landsat 8.

<br>

## Index

* [Import Dependencies and Connect to the Data Cube](#Urbanization_Using_NDBI_import)
* [Choose Platform and Product](#Urbanization_Using_NDBI_plat_prod)
* [Define the Extents of the Analysis](#Urbanization_Using_NDBI_define_extents)
* [Load Data from the Data Cube](#Urbanization_Using_NDBI_retrieve_data)
* [Show RGB Representation of the Area](#Urbanization_Using_NDBI_rgb)
* [Urbanization Analysis](#Urbanization_Using_NDBI_analysis)

## <span id="Urbanization_Using_NDBI_import">Import Dependencies and Connect to the Data Cube [&#9652;](#Urbanization_Using_NDBI_top)</span>

In [ ]:
import sys
import os
sys.path.append(os.environ.get('NOTEBOOK_ROOT'))

import matplotlib.pyplot as plt
import xarray as xr

from utils.data_cube_utilities.dc_display_map import display_map
from utils.data_cube_utilities.dc_rgb import rgb

from utils.data_cube_utilities.urbanization import NDBI
from utils.data_cube_utilities.vegetation import NDVI
from utils.data_cube_utilities.dc_water_classifier import NDWI

from datacube.utils.aws import configure_s3_access
configure_s3_access(requester_pays=True)

import datacube
dc = datacube.Datacube()

## <span id="Urbanization_Using_NDBI_plat_prod">Choose Platform and Product [&#9652;](#Urbanization_Using_NDBI_top)</span>

In [ ]:
# Get available products
products_info = dc.list_products()

In [ ]:
print("LANDSAT 7 Products:")
products_info[["platform", "name"]][products_info.platform == "LANDSAT_7"]

In [ ]:
print("LANDSAT 8 Products:")
products_info[["platform", "name"]][products_info.platform == "LANDSAT_8"]

**Choose the platforms and products**

In [ ]:
# These are the platforms (satellites) and products (datacube sets) 
# used for this demonstration.

platform = 'LANDSAT_8'
product = 'ls8_usgs_sr_scene'
collection = 'c1'
level = 'l2'

## <span id="Urbanization_Using_NDBI_define_extents">Define the Extents of the Analysis [&#9652;](#Urbanization_Using_NDBI_top)</span>

In [ ]:
# Kumasi, Ghana
# lat = (6.597724,6.781856)
# lon = (-1.727843,-1.509147)

# Accra, Ghana
lat = (5.5162, 5.6338)
lon = (-0.2657, -0.1373)

time_range = ("2019-01-01", "2019-12-31")

**Visualize the selected area**

In [ ]:
display_map(lat, lon)

## <span id="Urbanization_Using_NDBI_retrieve_data">Load Data from the Data Cube [&#9652;](#Urbanization_Using_NDBI_top)</span>

In [ ]:
desired_bands = ['red','green','nir','swir1', 'swir2', 'pixel_qa']  # needed by ndvi, ndwi, ndbi and cloud masking
desired_bands = desired_bands + ['blue'] # blue is needed for a true color visualization purposes

landsat_ds = dc.load(product = product,
                     platform = platform,
                     lat = lat,
                     lon = lon,
                     time = time_range,
                     measurements = desired_bands,
                     dask_chunks={'time':1, 'latitude':1000, 'longitude':1000})

In [ ]:
from utils.data_cube_utilities.clean_mask import landsat_clean_mask_full

clean_mask = landsat_clean_mask_full(dc, landsat_ds, product=product, platform=platform, 
                                     collection=collection, level=level)

In [ ]:
landsat_ds = landsat_ds.where(clean_mask)

## <span id="Urbanization_Using_NDBI_rgb">Show RGB Representation of the Area [&#9652;](#Urbanization_Using_NDBI_top)</span>

In [ ]:
median_composite = landsat_ds.median('time')

In [ ]:
plt.figure(figsize=(8,8))
median_composite[['red', 'green', 'blue']].to_array().plot.imshow(vmin=0, vmax=2500)
plt.show()

## <span id="Urbanization_Using_NDBI_analysis">Urbanization Analysis [&#9652;](#Urbanization_Using_NDBI_top)</span>

> **NDWI, NDVI, NDBI**  
You will very rarely have urban classification and water classifications apply to the same pixel. For urban analysis, it may make sense to compute not just urban classes, but classes that are unlikely to co-occur with urbanization, such as vegetation (e.g. NDVI) or water (e.g. NDWI).

In [ ]:
ndbi = NDBI(median_composite) # Urbanization
ndvi = NDVI(median_composite) # Dense Vegetation
ndwi = NDWI(median_composite) # High Concentrations of Water

In [ ]:
plt.figure(figsize=(8,8))
ndvi.plot(cmap = "Greens")
plt.show()

In [ ]:
plt.figure(figsize=(8,8))
ndwi.plot(cmap = "Blues")
plt.show()

In [ ]:
plt.figure(figsize=(8,8))
ndbi.plot(cmap = "Reds")
plt.show()

> **Merge into one Dataset**  
> If your data-arrays share the same set of coordinates, and you feel that you'll be using these values together in the future, you should consider merging them into an `xarray.Dataset`.

In [ ]:
urbanization_dataset = xr.merge((ndvi.rename('NDVI'), ndwi.rename('NDWI'), ndbi.rename('NDBI')))

In [ ]:
urbanization_dataset

>**Building a False Color Composite**  
> If you have three lowly correlated measurements, place the measurement on red, green, and blue channels and visualize them. 

In [ ]:
plt.figure(figsize=(8,8))
urbanization_dataset[["NDBI", "NDVI", "NDWI"]].to_array().plot.imshow(vmin=0, vmax=1)
plt.show()

>**Analyze The False Color Image**  
> Values that adhere strongly to individual classes adhere to their own color channel. In this example, NDVI adheres to green, NDWI adheres to blue, and NDBI adheres to red.  

> **Validate urbanization using other imagery**  
> Double check results using high-resolution imagery.  Compare to the false color mosaic
<br>

In [ ]:
display_map(latitude = lat ,longitude = lon)  